In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pylab
import scipy.io as sio
import scipy
from sklearn import neighbors
#%matplotlib inline

In [8]:
def LDE(data, label, k, d, t):
    '''
    data is the original data. m*n(m=samples, n=dimensions)
    '''
    data = np.matrix(data)
    N = data.shape[0]
    Dim = data.shape[1]
    
    # step 1
    # find the nearest neighbors (k). Here we get the distance from (x-y)^2
    # distance N*N
   
    data_tmp = np.sum(np.multiply(data, data), axis=1)
    distance=np.mat(data_tmp + data_tmp.T - 2*data*data.T)
    
    # the all neibors N
    neibors = np.argsort(distance,axis=1)
    
    
    # step 2
    # weights
    
    W=np.zeros([N,N])
    D=np.zeros([N,N])
    K=0
    W_=np.zeros([N,N])
    D_=np.zeros([N,N])
    K_=0
    for i in range(N):
        for j in neibors[i].A[0]:
            if(K<k and label[j]==label[i]):
                W[i, j] = math.exp(-distance[i, j]/t)
                W[j, i] = math.exp(-distance[i, j]/t) 
                K+=1
            elif(K_<k and label[j]!=label[i]):
                W_[i, j] = math.exp(-distance[i, j]/t)
                W_[j, i] = math.exp(-distance[i, j]/t) 
                K_+=1
            elif(K==k and K_==k):
                break
            else:
                continue
    
    for i in range(N):
        for j in range(N):
            D[i, i] +=W[i, j] 
            D_[i, i] +=W_[i, j]
    
    # Step 3
    # mappings 
    
    L=D-W
    L_=D_-W_
    eigenValues, eigenVectors=scipy.linalg.eig(np.dot(np.dot(data^T, L_), data), np.dot(np.dot(data^T, L)))
    eigenValuesSort=np.argsort(-eigenValues)
    mapping = eigenVectors[:, eigenValuesIndex[0:d]].T
    
    return mapping

In [10]:
data=sio.loadmat("Yale_32x32.mat")
face = data['fea']
label = data['gnd']
label = np.squeeze(label)
print(face.shape)
print(label.shape)

(165, 1024)
(165,)
